In [46]:
import pandas as pd
import pyodbc
import sqlite3
from datetime import datetime

# iedereen heeft andere servername btw

In [47]:
servername = 'LAPTOP-C1FMPSTV\\SQLEXPRESS01'

# Northwind connectie

In [48]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [49]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [50]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


# Dataframe maken

AenC

In [51]:
aenctest1 = pd.read_sql_query('SELECT emp_ID, city, state, start_date, sex, birth_date FROM employee', aenc_conn)
aenctest2 = pd.read_sql_query('SELECT state_id, country FROM state', aenc_conn)
aencmerged_df = pd.merge(aenctest1, aenctest2, left_on= 'state', right_on= 'state_id')
aencmerged_df.drop(columns=['state_id'], inplace=True)
aencmerged_df = aencmerged_df.rename(columns={'emp_ID':'employeeid'})
aencmerged_df.employeeid = aencmerged_df.employeeid.astype(int)

aencmerged_df['Source'] = "AC"
aencmerged_df

C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\4201798115.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest1 = pd.read_sql_query('SELECT emp_ID, city, state, start_date, sex, birth_date FROM employee', aenc_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\4201798115.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest2 = pd.read_sql_query('SELECT state_id, country FROM state', aenc_conn)


,employeeid,city,state,start_date,sex,birth_date,country,Source
0,102,Needham,MA,26-Feb-1987 12:00:00 AM,F,05-Jun-1959 12:00:00 AM,USA,AC
1,105,Waltham,MA,02-Jul-1987 12:00:00 AM,M,04-Dec-1961 12:00:00 AM,USA,AC
2,129,Atlanta,GA,04-Aug-1998 12:00:00 AM,M,30-Oct-1967 12:00:00 AM,USA,AC
3,148,Winchester,MA,04-Oct-1997 12:00:00 AM,F,13-Dec-1952 12:00:00 AM,USA,AC
4,160,Milton,MA,16-Dec-1987 12:00:00 AM,M,13-May-1948 12:00:00 AM,USA,AC
...,...,...,...,...,...,...,...,...
70,1643,Burlington,MA,15-Jun-1997 12:00:00 AM,F,12-Sep-1969 12:00:00 AM,USA,AC
71,1658,Waltham,MA,28-Aug-1997 12:00:00 AM,M,18-Jan-1974 12:00:00 AM,USA,AC
72,1684,Waltham,MA,13-Sep-1997 12:00:00 AM,F,31-Oct-1970 12:00:00 AM,USA,AC
73,1740,Bedford,MA,23-Dec-1997 12:00:00 AM,M,19-Jun-1966 12:00:00 AM,USA,AC


AdventureWorks

In [52]:
awtest1 = pd.read_sql_query('SELECT businessEntityID, JobTitle, BirthDate, Gender, HireDate FROM HumanResources.Employee', adventureworks_conn)
awtest2 = pd.read_sql_query('SELECT BusinessEntityID, AddressID FROM Person.BusinessEntityAddress', adventureworks_conn)
awtest3 = pd.read_sql_query('SELECT AddressID, StateProvinceID, City FROM Person.Address', adventureworks_conn)
awtest4 = pd.read_sql_query('SELECT StateProvinceID, CountryRegionCode FROM Person.StateProvince', adventureworks_conn)

awmerged_df1 = pd.merge(awtest1, awtest2, left_on='businessEntityID', right_on='BusinessEntityID').drop(columns=['BusinessEntityID'])
awmerged_df2 = pd.merge(awmerged_df1, awtest3, on='AddressID')
awmerged_df3 = pd.merge(awmerged_df2, awtest4, on='StateProvinceID')
awmerged_df3 = awmerged_df3.rename(columns={'businessEntityID':'employeeid'})
awmerged_df3 = awmerged_df3.rename(columns={'CountryRegionCode':'country'})

awmerged_df3['Source'] = "AW"
awmerged_df3

C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\2737852508.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest1 = pd.read_sql_query('SELECT businessEntityID, JobTitle, BirthDate, Gender, HireDate FROM HumanResources.Employee', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\2737852508.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest2 = pd.read_sql_query('SELECT BusinessEntityID, AddressID FROM Person.BusinessEntityAddress', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\2737852508.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection

,employeeid,JobTitle,BirthDate,Gender,HireDate,AddressID,StateProvinceID,City,country,Source
0,1,Chief Executive Officer,1969-01-29,M,2009-01-14,249,79,Newport Hills,US,AW
1,2,Vice President of Engineering,1971-08-01,F,2008-01-31,293,79,Renton,US,AW
2,3,Engineering Manager,1974-11-12,M,2007-11-11,224,79,Redmond,US,AW
3,4,Senior Tool Designer,1974-12-23,M,2007-12-05,11387,36,Minneapolis,US,AW
4,5,Design Engineer,1952-09-27,F,2008-01-06,190,79,Bellevue,US,AW
...,...,...,...,...,...,...,...,...,...,...
285,286,Sales Representative,1977-02-14,F,2013-05-30,36,77,Melbourne,AU,AW
286,287,European Sales Manager,1957-09-20,F,2012-04-16,300,79,Renton,US,AW
287,288,Sales Representative,1975-07-09,F,2013-05-30,35,20,Berlin,DE,AW
288,289,Sales Representative,1968-03-17,F,2012-05-30,37,14,Cambridge,GB,AW


NorthWind

In [53]:
nwtest = pd.read_sql_query('SELECT EmployeeID, Title, HireDate, City, Region, Country, BirthDate FROM employees', northwind_conn)
nwtest = nwtest.rename(columns={'EmployeeID':'employeeid'})
nwtest['Source'] = "NW"
nwtest

C:\Users\sheng\AppData\Local\Temp\ipykernel_53944\2183483448.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT EmployeeID, Title, HireDate, City, Region, Country, BirthDate FROM employees', northwind_conn)


,employeeid,Title,HireDate,City,Region,Country,BirthDate,Source
0,1,Sales Representative,1992-05-01,Seattle,WA,USA,1948-12-08,NW
1,2,"Vice President, Sales",1992-08-14,Tacoma,WA,USA,1952-02-19,NW
2,3,Sales Representative,1992-04-01,Kirkland,WA,USA,1963-08-30,NW
3,4,Sales Representative,1993-05-03,Redmond,WA,USA,1937-09-19,NW
4,5,Sales Manager,1993-10-17,London,None,UK,1955-03-04,NW
5,6,Sales Representative,1993-10-17,London,None,UK,1963-07-02,NW
6,7,Sales Representative,1994-01-02,London,None,UK,1960-05-29,NW
7,8,Inside Sales Coordinator,1994-03-05,Seattle,WA,USA,1958-01-09,NW
8,9,Sales Representative,1994-11-15,London,None,UK,1966-01-27,NW


Samenvoegen

In [54]:
employee = pd.concat([ nwtest, awmerged_df3, aencmerged_df], axis =0)

employee['row_id'] = range(len(employee))

employee['title'] = employee['Title'].combine_first(employee['JobTitle'])
employee['gender'] = employee['Gender'].combine_first(employee['sex'])
employee['city'] = employee['City'].combine_first(employee['city'])
employee['region'] = employee['Region']
employee['country'] = employee['Country'].combine_first(employee['country'])
employee['hiredate'] = employee['HireDate'].combine_first(employee['start_date'])
employee['birthdate'] = employee['BirthDate'].combine_first(employee['birth_date'])
employee['source'] = employee['Source']


final_columns = [
    'employeeid', 'title', 'gender', 'city', 'region', 'country', 
    'hiredate', 'birthdate', 'row_id', 'source'
]

final_employee_df = employee[final_columns]

employee = final_employee_df



In [55]:
employee.dtypes
#test

employeeid     int64
title         object
gender        object
city          object
region        object
country       object
hiredate      object
birthdate     object
row_id         int64
source        object
dtype: object

# DATABASE

In [56]:
united_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')
unitedCursor = united_conn.cursor()
unitedCursor

In [57]:
# Convert all date columns to a uniform datetime format
employee['hiredate'] = pd.to_datetime(employee['hiredate'])
employee['birthdate'] = pd.to_datetime(employee['birthdate'])

# Specify the desired format
date_format = '%d-%m-%Y'

# Apply the format to the datetime columns
employee['hiredate'] = employee['hiredate'].dt.strftime(date_format)
employee['birthdate'] = employee['birthdate'].dt.strftime(date_format)

# Resultaat bekijke
print(employee.head())

   employeeid                  title gender      city region country  \
0           1   Sales Representative    NaN   Seattle     WA     USA   
1           2  Vice President, Sales    NaN    Tacoma     WA     USA   
2           3   Sales Representative    NaN  Kirkland     WA     USA   
3           4   Sales Representative    NaN   Redmond     WA     USA   
4           5          Sales Manager    NaN    London   None      UK   

     hiredate   birthdate  row_id source  
0  01-05-1992  08-12-1948       0     NW  
1  14-08-1992  19-02-1952       1     NW  
2  01-04-1992  30-08-1963       2     NW  
3  03-05-1993  19-09-1937       3     NW  
4  17-10-1993  04-03-1955       4     NW  


In [58]:
employee = employee.where(pd.notna(employee), None)
employee

,employeeid,title,gender,city,region,country,hiredate,birthdate,row_id,source
0,1,Sales Representative,None,Seattle,WA,USA,01-05-1992,08-12-1948,0,NW
1,2,"Vice President, Sales",None,Tacoma,WA,USA,14-08-1992,19-02-1952,1,NW
2,3,Sales Representative,None,Kirkland,WA,USA,01-04-1992,30-08-1963,2,NW
3,4,Sales Representative,None,Redmond,WA,USA,03-05-1993,19-09-1937,3,NW
4,5,Sales Manager,None,London,None,UK,17-10-1993,04-03-1955,4,NW
...,...,...,...,...,...,...,...,...,...,...
70,1643,None,F,Burlington,None,USA,15-06-1997,12-09-1969,369,AC
71,1658,None,M,Waltham,None,USA,28-08-1997,18-01-1974,370,AC
72,1684,None,F,Waltham,None,USA,13-09-1997,31-10-1970,371,AC
73,1740,None,M,Bedford,None,USA,23-12-1997,19-06-1966,372,AC


In [59]:
from datetime import datetime
for index, row in employee.iterrows():
    date_string = [row['hiredate'],row['birthdate']]
    for dateString in date_string:
        date_parts = dateString.split("-")
        date_obj = datetime.strptime(dateString, '%d-%m-%Y')
        date_value = date_obj.strftime('%Y-%m-%d')
        # Extract year, month, and day
        year = int(date_parts[2])
        month = int(date_parts[1])
        day = int(date_parts[0])
        try:
            query = """INSERT INTO date_table (
                DATE_date,
                DATE_year_nr, 
                DATE_month_nr, 
                DATE_day_nr
            ) VALUES (?, ?, ?, ?)"""  
            
            values = (
                date_value, 
                year,
                month,
                day  
            )

            unitedCursor.execute(query, values)

        except pyodbc.IntegrityError as e:
            continue
        except pyodbc.Error as e:
            print("Er is een fout opgetreden: ", e)
            print("Foutieve query: ", query)
        
        
unitedCursor.commit()

In [60]:
employee = employee.replace({'': None})
employee.where(pd.notna(employee), None)
for index, row in employee.iterrows():
    date_string = row['hiredate']
    date_obj = datetime.strptime(date_string, '%d-%m-%Y')
    date_value = date_obj.strftime('%Y-%m-%d')
    date_string2 = row['birthdate']
    date_obj2 = datetime.strptime(date_string2, '%d-%m-%Y')
    date_value2 = date_obj2.strftime('%Y-%m-%d')
    try:
        query = """INSERT INTO employee (
            employee_employeeid,
            employee_title,
            employee_gender, 
            employee_city,
            employee_region,
            employee_country,
            employee_hiredate,
            employee_birthdate,
            employee_source
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"""  
        
        values = (
            row['employeeid'],
            row['title'],
            row['gender'],
            row['city'],
            row['region'],
            row['country'],
            date_value,
            date_value2,
            row['source']
        )

        unitedCursor.execute(query, values)

    except pyodbc.IntegrityError as e:
        continue
    except pyodbc.Error as e:
        print("Er is een fout opgetreden: ", e)
        print("Foutieve query: ", query)
    
unitedCursor.commit()

print("Data is succesvol geïmporteerd in de UnitedOutdoors database")

Data is succesvol geïmporteerd in de UnitedOutdoors database
